In [1]:
import numpy as np
import sympy as sy

In [2]:
M1 = np.array([[0.2,0.2,0.6],
               [0.5,0.1,0.4],
               [0.3,0.7,0.0]
              ])
M2 = np.array([[0.2,0.2,0.6,0.0,0.0,0.0,0.0,0.0],
               [0.5,0.1,0.4,0.0,0.0,0.0,0.0,0.0],
               [0.3,0.7,0.0,0.0,0.0,0.0,0.0,0.0],
               [0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0],
               [0.0,0.0,0.0,0.3,0.7,0.0,0.0,0.0],
               [0.0,0.0,0.0,0.0,0.0,0.2,0.7,0.1],
               [0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.6],
               [0.0,0.0,0.0,0.0,0.0,0.5,0.2,0.3]
              ])
mu_0 = np.array([[1],[0],[0],[0],[0],[0],[0],[0]])

In [3]:
def gauss_jordan_exact(A):
    # Simplification exacte des éléments de la matrice A
    A = np.array([[sy.nsimplify(A[i, j]) for j in range(len(A[i]))] for i in range(len(A))])

    # Boucle principale pour effectuer l'algorithme de Gauss-Jordan
    for i in range(len(A)):
        # Échange de lignes si l'élément diagonal est nul
        if A[i, i] == 0:
            # Chercher une ligne non nulle en dessous
            j = i + 1
            # Si une ligne non nulle est trouvée en dessous
            while (j < len(A)) and (A[j, i] == 0):
                j += 1
                 # Ajouter la ligne trouvée à la ligne actuelle pour éviter le 0 sur la diagonale            if j < len(A):
                A[i, :] = A[i, :] + A[j, :]
            else:
                continue

        # Normalisation de la ligne pour que l'élément diagonal soit égal à 1
        c = A[i, i]
        A[i, :] = A[i, :] / c

        # Élimination des autres éléments dans la colonne
        for j in range(len(A)):
            if j != i:
                c = -A[j, i] / A[i, i] # le coef qui annule le coeff de la ligne j dans la colonne i
                A[j, :] = A[j, :] + c * A[i, :] # On ajoute a la ligne j la ligne i * par le coeff

    # Retourne la matrice résultante après l'algorithme de Gauss-Jordan
    return A


def mesure_in_list(mu,l,ecart=0.01):
    """Détermine si une mesure appartient à un liste avec une tolérance de écart."""
    dedans = False
    for j in range(len(l)):
        dedans = dedans | np.isclose(l[j],mu,atol=ecart).all()
    return dedans

# Exercice 1

In [4]:
def is_stochastique(M):
    return (M.shape[0] == M.shape[1]) and ((M <= 1).all()) and ((M.sum(axis=1) == 1).all())

In [5]:
is_stochastique(M2)

np.True_

In [6]:
is_stochastique(M1)

np.True_

# Exercice 2

In [7]:
def mesure_asymptotique_emp(P, mu):
    # Transposer la matrice P
    P = P.T

    # Calculer le produit matriciel P * mu
    mu_temp = P.dot(mu)

    # Répéter le processus jusqu'à ce que mu soit suffisamment proche de mu_temp
    while not np.isclose(mu, mu_temp, atol=10**(-3)).all():
        # Copier mu_temp dans mu
        mu = mu_temp.copy()

        # Mettre à jour mu_temp avec le nouveau produit matriciel P * mu
        mu_temp = P.dot(mu)

    # Retourner mu arrondi à trois décimales
    return np.round(mu, 3)


In [8]:
mesure_asymptotique_emp(M2,mu_0)

array([[0.333],
       [0.333],
       [0.334],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ],
       [0.   ]])

# Exercice 3

In [9]:
def mesures_invariantes_emp(P):
    """
    Détermine l'ensemble des mesures asymptotiques d'une chaîne
    de Markov pour toutes les mesures de la forme (0 ... 0 1 0 ... 0)
    c'est-à-dire un 1 et des 0.
    """
    # Liste pour stocker les mesures asymptotiques
    l_inv = []

    # Parcours de toutes les lignes de la matrice P
    for i in range(len(P)):
        # Initialisation d'un vecteur mu avec un 1 à la position i et des 0 ailleurs
        mu = np.zeros((len(P), 1))
        mu[i] = 1

        # Calcul de la mesure asymptotique empirique pour le vecteur mu
        mu = mesure_asymptotique_emp(P, mu)

        # Vérification si la mesure mu est déjà dans la liste l_inv avec une tolérance de 0.01
        if not mesure_in_list(mu, l_inv, 0.01):
            # Ajout de la mesure à la liste l_inv
            l_inv.append(mu)

    # Retour de l'ensemble des mesures asymptotiques
    return l_inv


In [10]:
mesures_invariantes_emp(M2)

[array([[0.333],
        [0.333],
        [0.334],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.   ]]),
 array([[0.   ],
        [0.   ],
        [0.   ],
        [0.376],
        [0.624],
        [0.   ],
        [0.   ],
        [0.   ]]),
 array([[0.   ],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.36 ],
        [0.316],
        [0.324]])]

# Exerice 4

In [11]:
def mesure_inv_irr(M):
    """
    Calcule une mesure invariante pour une matrice de transition irréductible.
    """
    # Résoudre le système d'équations linéaires en utilisant la méthode de Gauss-Jordan
    # np.eye(M.shape[0]): Création d'une matrice identité de même taille que la matrice de transition M. Cela est nécessaire car le système d'équations à résoudre est (M^T - I)x = 0, où M^T est la matrice de transition et I est la matrice identité.
    # [:, -1]: Sélection de la dernière colonne de la matrice résultante après l'application de l'algorithme de Gauss-Jordan. Cette colonne contient les solutions du système d'équations linéaires.
    # -: Les solutions sont multipliées par -1. Cela est probablement nécessaire en fonction du côté gauche du système d'équations (M^T - I)x = 0, où le terme de droite est généralement négatif.
    sol = - gauss_jordan_exact(M.T - np.eye(M.shape[0]))[:, -1]

    # Ajuster la dernière composante pour garantir que la somme soit égale à 1
    sol[-1] = 1

    # Normaliser le vecteur pour qu'il représente une mesure de probabilité valide
    sol = sol / sol.sum()

    # Retourner la mesure invariante
    return sol


In [12]:
mesure_inv_irr(M1)

array([1/3, 1/3, 1/3], dtype=object)

In [13]:
mesures_invariantes_emp(M1)

[array([[0.333],
        [0.333],
        [0.334]])]

# Exercice 5

In [14]:
def mesures_invariantes(M):
    # Initialisation de la liste pour stocker les mesures invariantes
    l = []

    # Résolution du système d'équations linéaires avec l'algorithme de Gauss-Jordan
    sol = gauss_jordan_exact(M.T - np.eye(M.shape[0]))

    # Boucle pour parcourir chaque ligne de la matrice
    for i in range(len(M)):
        # Vérification si l'élément diagonal de la solution est nul
        if sol[i, i] == 0:
            # Construction du vecteur mu correspondant à la mesure invariante
            mu = -sol[:, i].copy()
            mu[i] = 1
            mu = mu / mu.sum()

            # Ajout du vecteur mu à la liste des mesures invariantes
            l.append(mu)

    # Retour de la liste des mesures invariantes
    return l


In [15]:
mesures_invariantes(M1)

[array([1/3, 1/3, 1/3], dtype=object)]

In [16]:
mesures_invariantes_emp(M1)

[array([[0.333],
        [0.333],
        [0.334]])]

In [17]:
mesures_invariantes(M2)

IndexError: index 8 is out of bounds for axis 0 with size 8

In [20]:
mesures_invariantes_emp(M2)

[array([[0.333],
        [0.333],
        [0.334],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.   ]]),
 array([[0.   ],
        [0.   ],
        [0.   ],
        [0.376],
        [0.624],
        [0.   ],
        [0.   ],
        [0.   ]]),
 array([[0.   ],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.   ],
        [0.36 ],
        [0.316],
        [0.324]])]